# manoeuvring model Bias-variance tradeoff

The accelerations ($y$) obtained from the model tests are generated by the unknown real model $f(x)$ and the irreducible error $\epsilon \sim \mathcal{N}(0,\sigma^2)$ .

$$
y = f(x) + \epsilon
$$ (eq_y)

In the manoeuvring models $\hat{f}(x)$ (predictor) approximates the true model $f(x)$ 

$$
y = \hat{f}(x) + \epsilon_G
$$ (eq_y)

The predictor is fitted on a dataset $D$ (the model tests).

$$
D = \{(x_1,y_1)..(x_n,y_n) \}
$$ (eq_D)

$$
y = \hat{f}_D(x) + \epsilon_G
$$ (eq_y)

For the predictor to be usefull it must perform well on unseen data so that it generalizes to $x$ values outside the training dataset $D$. The predictor will then have a generalization error which can be decomposed in bias error, variance and irreducible error. The bias error describes how well the predictor $\hat{f}(x)$ approximates the real model ${f}(x)$. The variance describes the robustness of the predictor, how much it changes with different training datasets. The irreducible error origins from inherent noise in the problem itself and is an error term that will always be present.

A simple predictor simplifies the real model too much giving a high bias error. A simple model is not so sensitive to the training data and are therefore robust and has a similar performance regardless of the training dataset which giving a low variance. 
A more complex model acts more like the real model giving lower bias but may also have a high variance if the model generalizes very badly to data outside the training dataset.
Very simple models underfitts the training data and the too complex models overfitts it.  
The goal is to create a predictor with both low bias error and low variance which usually is a tradeoff between the two.

The expected error of $\hat{f}(x)$ on unseen sample $x$ can be decomposed as:

$$
\mathop{\mathbb{E_D}} \left[ (y-\hat{f}_D(x))^2 \right] = MSE = \left( Bias_D[\hat{f}(x)]\right)^2 +Var_D[\hat{f}(x)] + \sigma^2
$$ (eq_bias_variance)

$$
Bias_D[\hat{f}(x)] = \mathop{\mathbb{E_D}} \left[\hat{f}_D(x) \right] - f(x)
$$ (eq_bias)

$$
Var_D[\hat{f}(x)] = \mathop{\mathbb{E_D}} \left[ (\mathop{\mathbb{E_D}} [\hat{f}_D(x)] - \hat{f}_D(x))^2 \right] 
$$ (eq_variance)


Since the real model $f(x)$ is not known the expected bias is calculated from the observed values $y$

$$
Bias_D[\hat{f}(x)] = \mathop{\mathbb{E_D}} \left[\hat{f}_D(x) \right] - y
$$ (eq_bias)


To find the best bias-variance tradeoff the data is first divided into a training set and a test set. The training set is further divided into sub training sets. Predictions of values in the test set is then performed with models fitted to the sub training sets. The bias and variance can then be calculated for each manoeuvring model.

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)

from vessel_manoeuvring_models.visualization.plot import plot, track_plots
ship_data = catalog.load("ship_data")


In [ ]:
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
from vessel_manoeuvring_models.bias_variance_tradeoff import (train_test_split, 
                                        train_test_split_run, 
                                        train_test_split_exteme, 
                                        train_predict, 
                                        pivot_mean, 
                                        variances,
                                        expected,
                                        errors,
                                        error_bars)

In [ ]:
join = "joined"
data = catalog.load(f"{ join }.data_ek_smooth")

added_masses = catalog.load("added_masses")
ship_data = catalog.load("ship_data")
exclude_parameters = catalog.load("params:motion_regression.exclude_parameters")

runs_meta_data = catalog.load("runs_meta_data")
runs_meta_data.sort_values(by='description', inplace=True)

vmms = {}
for vmm_name in vmm_names:
    vmms[vmm_name] = catalog.load(f"{vmm_name}")
    
regressions = {}
for vmm_name in vmm_names:
    regressions[vmm_name] = catalog.load(f"{ vmm_name }.motion_regression.joined.regression")


In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,5)

data_ = data.copy()
data_['beta'] = -np.arctan2(data_['v'],data['u'])
data_['beta_deg'] = np.rad2deg(data_['beta'])
data_['r_deg'] = np.rad2deg(data_['r'])
data_['delta_deg'] = np.rad2deg(data_['delta'])

mask = runs_meta_data.index.isin(data_['id'].unique())
runs_meta_data_selected = runs_meta_data.loc[mask].copy()
mask = runs_meta_data_selected.duplicated(subset='description', keep='last')
runs_meta_data_selected = runs_meta_data_selected.loc[~mask]
data_groups = data_.groupby(by='id')

for id, meta_data in runs_meta_data_selected.iterrows():
    
    if not id in data_groups.groups.keys():
        continue
    
    df_ = data_groups.get_group(id)
    description = runs_meta_data.loc[id]['description']
    df_.plot(x='beta_deg', y='r_deg', ax=ax, label=description)
    
ax.set_ylabel(r'$r$ (yaw rate) $[deg/s]$')
ax.set_xlabel(r'$\beta$ (drift angle) $[deg]$');

fig_name = f"fig_beta_vs_r"
glue(fig_name, fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,5)

for id, meta_data in runs_meta_data_selected.iterrows():
    
    if not id in data_groups.groups.keys():
        continue
    
    df_ = data_groups.get_group(id)
    description = runs_meta_data.loc[id]['description']
    df_.plot(x='delta_deg', y='u', ax=ax, label=description)
    
ax.set_ylabel(r'$u$ (speed) $[m/s]$')
ax.set_xlabel(r'$\delta$ (rudder angle) $[deg]$');

fig_name = f"fig_delta_vs_u"
glue(fig_name, fig, display=False)

Selecting which part of the data that should be in the test set can be done in many ways, depending on how the model should generalize to data outside the training data. The drift angles, yaw rates, rudder angles and speed are shown in {numref}```fig_beta_vs_r``` and {numref}```fig_delta_vs_u```. It can be seen that the Turning circle test is very different from the other tests in terms of: maximum drift and rudder angles as well as the minimum speed. 

```{glue:figure} fig_beta_vs_r
:figwidth: 600px
:name: "fig_beta_vs_r"
Drift angle and yaw rate from all the model tests.
```

```{glue:figure} fig_delta_vs_u
:figwidth: 600px
:name: "fig_delta_vs_u"
Drift angle and yaw rate from all the model tests.
```

The bias-variance tradeoff will be investigated for three scenarios having the three differen test sets:
* Random test set.
* Extreme values as test set

In [ ]:
regression = regressions['vmm_martins_simple']

## Random test set

In [ ]:
X_train, y_train, X_test, y_test, train_data = train_test_split(X=regression.X_N,
                                                                y=regression.y_N, 
                                                                test_ratio=0.01)


fig,ax=plt.subplots()
y_train.plot(ax=ax, label='train', style='.')
y_test.plot(ax=ax, label='test', style='.')
ax.set_xlabel('sample')
ax.set_ylabel('$y$')
ax.legend();

fig_name = "fig_test_split_random"
glue(fig_name, fig, display=False)

{numref}```fig_test_split_random``` show how the data (yaw acceleration in this case) from all the model tests have been randomly split into a training set and a test set.


```{glue:figure} fig_test_split_random
:figwidth: 600px
:name: "fig_test_split_random"
Random train test split for the yaw acceleration data.
```

In [ ]:
def train_predict_vmms_random(dof='N'):

    df_sample_predictions = pd.DataFrame()
    
    for vmm_name, regression in regressions.items():
        
        X_name = f"X_{dof}"
        y_name = f"y_{dof}"
        X = getattr(regression, X_name)
        y = getattr(regression, y_name)
        
        X_train, y_train, X_test, y_test, train_data = train_test_split(X=X,
                                                                        y=y, 
                                                                        test_ratio=0.01)
        
        df_ = train_predict(train_data, X_test=X_test, y_test=y_test, train_ratio=0.005, N_trainings=50)
        df_['vmm'] = vmm_name
        df_sample_predictions = df_sample_predictions.append(df_, ignore_index=True)
        
    df_sample_predictions.sort_values(by=['parameters','x'], inplace=True)
    
    df_sample_predictions['residual'] = df_sample_predictions['y_hat'] - df_sample_predictions['z']
    df_sample_predictions['residual^2'] = df_sample_predictions['residual']**2
    return df_sample_predictions,y_test

In [ ]:
%%time

for dof in ['X','Y','N']:
    
    df_sample_predictions_random, y_test = train_predict_vmms_random(dof=dof)
    df_errors_random = errors(df_sample_predictions=df_sample_predictions_random, y_test=y_test)
    
    if dof=='X':
        smart_scale = True
    else:
        smart_scale = False
    
    fig = error_bars(df_errors_random, smart_scale=smart_scale)
    fig_name = f"fig_error_bars_random_{dof}"
    glue(fig_name, fig, display=False)   
    display(df_errors_random)

The generalization error expressed as the Mean Square Error MSE from prediction on the random test set with many models trained on subsets of the training set for the surge, sway and yaw regressions are shown in {numref}```fig_error_bars_random_X```, {numref}```fig_error_bars_random_X``` and {numref}```fig_error_bars_random_N```.

The errors have been decomposed into bias and variance. The manoeuvring models are ordered by their complexity (number of parameters) with the linear model being the simplest and the Abkowitz model being the most complex. It can be seen that the Abkowitz model has a large variance that contributes to the error.

```{glue:figure} fig_error_bars_random_X
:figwidth: 600px
:name: "fig_error_bars_random_X"
Error bars for surge acceleration regressions with random test set.
```

```{glue:figure} fig_error_bars_random_Y
:figwidth: 600px
:name: "fig_error_bars_random_Y"
Error bars for sway acceleration regressions with random test set.
```

```{glue:figure} fig_error_bars_random_N
:figwidth: 600px
:name: "fig_error_bars_random_N"
Error bars for yaw acceleration regressions with random test set.
```

## Extreme test set

In [ ]:
X_train, y_train, X_test, y_test, train_data = train_test_split_exteme(X=regression.X_N, y=regression.y_N, data=data, 
                                                                          min_ratio=0.05, 
                                                                          max_ratio=0.05,
                                                                          min_keys=['u'],
                                                                          max_keys=['v','r','delta'])


fig,ax=plt.subplots()

y_train.plot(ax=ax, label='train', style='.')
y_test.plot(ax=ax, label='test', style='.')

index = data.groupby(by='id').get_group(22774).index
ax.fill_between(index, y1=regression.y_N.max(), y2=regression.y_N.min(), color='lightgrey', label='Turning Circle')

ax.set_xlabel('sample')
ax.set_ylabel('$y$')
ax.legend();

fig_name = "fig_test_split_extreme"
glue(fig_name, fig, display=False)

{numref}```fig_test_split_extreme``` show how the data (yaw acceleration in this case) from all the model tests have been split into a training set and a test set. The test set have been selected to contain all the extreme values from the model tests being the data with 5% largest absolute value of $v$, $r$ and $\delta$ and the 5% smallest absolute value of $u$. This reflects most of the data from the Turning circle test as indicated in {numref}```fig_test_split_extreme```.



```{glue:figure} fig_test_split_extreme
:figwidth: 600px
:name: "fig_test_split_extreme"
Random train test split for the yaw acceleration data.
```

In [ ]:
def train_predict_vmms_extreme(dof='N'):
    
    df_sample_predictions = pd.DataFrame()
    
    for vmm_name, regression in regressions.items():
        
        X_name = f"X_{dof}"
        y_name = f"y_{dof}"
        X = getattr(regression, X_name)
        y = getattr(regression, y_name)
                
        X_train, y_train, X_test, y_test, train_data = train_test_split_exteme(X=X,
                                                                            y=y,
                                                                            data=data,
                                                                            min_ratio=0.05, 
                                                                            max_ratio=0.05,
                                                                            min_keys=['u'],
                                                                            max_keys=['v','r'])
        
        df_ = train_predict(train_data, X_test=X_test, y_test=y_test, train_ratio=0.01, N_trainings=100)
        df_['vmm'] = vmm_name
        df_sample_predictions = df_sample_predictions.append(df_)
        
    df_sample_predictions.sort_values(by=['parameters','x'], inplace=True)
    
    df_sample_predictions['residual'] = df_sample_predictions['y_hat'] - df_sample_predictions['z']
    df_sample_predictions['residual^2'] = df_sample_predictions['residual']**2
    return df_sample_predictions, y_test

In [ ]:
%%time

for dof in ['X','Y','N']:
    
    df_sample_predictions_extreme, y_test = train_predict_vmms_extreme(dof=dof)
    df_errors_extreme = errors(df_sample_predictions=df_sample_predictions_extreme, y_test=y_test)
    
    if dof=='X':
        smart_scale = True
    else:
        smart_scale = False
    
    fig = error_bars(df_errors_extreme, smart_scale=smart_scale)
    fig_name = f"fig_error_bars_extreme_{dof}"
    glue(fig_name, fig, display=False)   
    
    display(df_errors_extreme)

The generalization error expressed as the Mean Square Error MSE from prediction on the extreme test set with many models trained on subsets of the training set for the surge, sway and yaw regressions are shown in {numref}```fig_error_bars_extreme_X```, {numref}```fig_error_bars_extreme_X```, {numref}```fig_error_bars_extreme_N```.

The trend with high variance for the Abkowitz model, being the most complex model, is even more pronounced when using the extreme test set.

```{glue:figure} fig_error_bars_extreme_X
:figwidth: 600px
:name: "fig_error_bars_extreme_X"
Error bars for surge acceleration regressions with extreme test set.
```

```{glue:figure} fig_error_bars_extreme_Y
:figwidth: 600px
:name: "fig_error_bars_extreme_Y"
Error bars for sway acceleration regressions with extreme test set.
```

```{glue:figure} fig_error_bars_extreme_N
:figwidth: 600px
:name: "fig_error_bars_extreme_N"
Error bars for yaw acceleration regressions with extreme test set.
```